<a href="https://colab.research.google.com/github/HarikaSuryam/HomeAssignments/blob/main/Assignment3_HarikaVinjamuri_700769273.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Home Assignment 3 **

Student name - Harika Vinjamuri

Student Id - 700769273

In [13]:
# Solution 1

# importing required libraries
import requests
import numpy as np
import tensorflow as tf


# Download The Little Prince
url = "https://www.gutenberg.org/files/118/118-0.txt"
response = requests.get(url)
text = response.text

# viewing first 100 chars
# print(text[:100])


# Unique characters
chars = sorted(set(text))
# print(f'Total unique chars: {len(chars)}')

# craete mapping from char to index and reverse
char_to_idx = {ch: i for i, ch in enumerate(chars)}
idx_to_char = {i: ch for ch, i in char_to_idx.items()}

# encode text
encoded = [char_to_idx[ch] for ch in text]
encoded = encoded[:10000] # reduced to improve the performance

# training sequences
seq_length = 50
steps = 1

sequences = []
next_chars = []

for i in range(0, len(encoded) - seq_length, steps):
    sequences.append(encoded[i:i+seq_length])
    next_chars.append(encoded[i+seq_length])



X = np.array(sequences)
y = np.array(next_chars)


model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(chars), 256, input_length=seq_length),
    tf.keras.layers.LSTM(256, return_sequences=False),
    tf.keras.layers.Dense(len(chars), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')
model.summary()


epochs = 5
batch_size = 128
history = model.fit(X, y, epochs=epochs, batch_size=batch_size)

# Text generation
def generate_text(start_string, length=500, temperature=1.0):
    """Generates text by sampling from the trained LSTM model."""
    input_indices = [char_to_idx[ch] for ch in start_string]
    text = start_string

    for _ in range(length):
        padded_input = np.pad(input_indices,
                                (seq_length - len(input_indices), 0),
                                mode='constant', constant_values=0)
        padded_input = padded_input.reshape(1, -1)
        preds = model.predict(padded_input, verbose=0)[0]

        preds /= temperature
        preds = np.exp(np.log(preds) / temperature)
        preds /= np.sum(preds)

        idx = np.random.choice(len(chars), p=preds)
        text += idx_to_char[idx]
        input_indices.append(idx)
        input_indices = input_indices[1:]  # slide forward

    return text


# Generate 500 chars of new text
generated = generate_text("To be or not to be ", length=500, temperature=0.5)
print("Generated text:")
print(generated)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
78/78 ━━━━━━━━━━━━━━━━━━━━ 48s 582ms/step - loss: 3.6399
Epoch 2/5
78/78 ━━━━━━━━━━━━━━━━━━━━ 82s 577ms/step - loss: 2.8136
Epoch 3/5
78/78 ━━━━━━━━━━━━━━━━━━━━ 81s 562ms/step - loss: 2.4286
Epoch 4/5
78/78 ━━━━━━━━━━━━━━━━━━━━ 44s 559ms/step - loss: 2.2772
Epoch 5/5
78/78 ━━━━━━━━━━━━━━━━━━━━ 82s 554ms/step - loss: 2.1441
Generated text:
To be or not to be the the nof of Formand the in the enting the the  Entet the the Elet wither alde the the enthe int alle wor to Ik the the Gut in the Woig of or and the the the  Dume and to the the the the the the the Domig of Guing of bith the et an tice in the the the aning the the the the to the the whe the the on the the the the net and sing in Foumed douter to cor the the do tres the wring of to noret on the nont intert in und the Laid fof out as wome the tho the te blet the in the the ep thang couly intion


The temperature controls the randomness in sampling the predicted character.

Higher temperature (>1): increases randomness; the sampling might produce more unusual or creative text, sometimes less coherent.

Lower temperature (<1): makes sampling more deterministic; it sticks close to the most likely character.

Temperature == 1: maintains the true predicted probability distribution.

In [15]:
# Solution 2

# import required libraries
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

# defining function
def nlp_preprocessing(sentence):
    # Tokenizing sentence into words
    tokens = word_tokenize(sentence)
    print("Original Tokens:")
    print(tokens)

    # Removing Stopwords
    stopwords_set = set(stopwords.words("english"))
    tokens_without_stopwords = [word for word in tokens if word.lower() not in stopwords_set]
    print("\nTokens Without Stopwords:")
    print(tokens_without_stopwords)

    # Perform Stemming
    stemmer = PorterStemmer()
    stemmed = [stemmer.stem(word) for word in tokens_without_stopwords]
    print("\nStemmed Words:")
    print(stemmed)


# apply the defined funaction to the below sentence
sentence = "NLP techniques are used in virtual assistants like Alexa and Siri."
nlp_preprocessing(sentence)


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Original Tokens:
['NLP', 'techniques', 'are', 'used', 'in', 'virtual', 'assistants', 'like', 'Alexa', 'and', 'Siri', '.']

Tokens Without Stopwords:
['NLP', 'techniques', 'used', 'virtual', 'assistants', 'like', 'Alexa', 'Siri', '.']

Stemmed Words:
['nlp', 'techniqu', 'use', 'virtual', 'assist', 'like', 'alexa', 'siri', '.']


**Stemming:**
Stemming cuts words down to their base form by removing suffixes.
Example: stemming “running” -> “run”

**Lemmatization:**
Lemmatization converts words to their base or dictionary form (lemma) while considering context and POS (part of speech).
Example: lemmatization “running” -> “run”

Remove stop words to reduce noise and improve algorithm performance in tasks like document clustering, information retrieval, or topic modeling — where high-frequency words carry little semantic weight.

Stop words can be crucial for understanding context and meaning, especially in sentiment analysis, dependency parsing, or machine translation, where their presence or absence can affect semantic relationships.

In [16]:
# Solution 3

import spacy

# Load english language model
nlp = spacy.load("en_core_web_sm")

# input
sentence = "Barack Obama served as the 44th President of the United States and won the Nobel Peace Prize in 2009."

# Process sentence with spaCy
doc = nlp(sentence)

# Printing extracted entities
for ent in doc.ents:
    print(f"Text: {ent.text}, Label: {ent.label_}, Start: {ent.start_char}, End: {ent.end_char}")



Text: Barack Obama, Label: PERSON, Start: 0, End: 12
Text: 44th, Label: ORDINAL, Start: 27, End: 31
Text: the United States, Label: GPE, Start: 45, End: 62
Text: the Nobel Peace Prize, Label: WORK_OF_ART, Start: 71, End: 92
Text: 2009, Label: DATE, Start: 96, End: 100


**NER (Named Entity Recognition)** focuses on identifying entities in the text, such as people, organizations, locations, dates, and more.

**POS (Part of Speech)** tagging, meanwhile, focuses on grammatical roles (noun, verb, adjectival, etc.).


**Real Time Applications for NER :**

Financial News: NER helps identify companies, people, and financial instruments in financial articles, which can aid in financial analysis and automated trading decisions.

Search Engines: NER helps search engines identify entities within search queries to provide more accurate and context-specific search results (for example, distinguishing “Amazon” the company from “Amazon” the rain forest).

In [19]:
# Solution 4

import numpy as np

def scaled_dot_product_attention(Q, K, V):
    """
    Perform scaled dot-product attention.

    Args:
        Q: Query matrix
        K: Key matrix
        V: Value matrix

    Returns:
        Attention weights and output after applying attention
    """
    d = K.shape[1]  # dimension of key

    # Calculating raw scores
    scores = np.dot(Q, K.T).astype(float)

    # Square root of d
    scores /= np.sqrt(d)

    # Applying softmax
    weights = np.exp(scores - np.max(scores, axis=-1, keepdims=True))
    weights /= np.sum(weights, axis=-1, keepdims=True)

    # Multiply weights
    output = np.dot(weights, V)
    return weights, output


# inputs
Q = np.array([
    [1, 0, 1, 0],
    [0, 1, 0, 1]
], dtype=float)

K = np.array([
    [1, 0, 1, 0],
    [0, 1, 0, 1]
], dtype=float)

V = np.array([
    [1, 2, 3, 4],
    [5, 6, 7, 8]
], dtype=float)


# Scaled dot-product
weights, output = scaled_dot_product_attention(Q, K, V)

print("Attention Weights:")
print(weights)
print("\nFinal Output:")
print(output)


Attention Weights:
[[0.73105858 0.26894142]
 [0.26894142 0.73105858]]

Final Output:
[[2.07576569 3.07576569 4.07576569 5.07576569]
 [3.92423431 4.92423431 5.92423431 6.92423431]]


**Division performed for attention score by underroot d in the scaled dot-product:**

We scale by the square root of the key dimension (d) to avoid the dot product growing too large with high dimensions. Large scores can push the softmax into saturated regimes, which can diminish the gradient and make training less effective. So scaling helps keep the values within a range where softmax operates gracefully.

**Self-attention** lets each word attend to all other words in the sentence. This way, the model can learn dependency relationships, context, and semantic connections, regardless of their positions in the sentence. This ability to combine information from the entire sentence is a key advantage for understanding context and generating coherent representations.

In [20]:
# !pip install transformers

In [24]:
# Solution 4
from transformers import pipeline

# Load sentiment analysis pipeline
sentiment_pipeline = pipeline("sentiment-analysis", model='distilbert-base-uncased-finetuned-sst-2-english')

# Input sentence
sentence = "Despite the high price, the performance of the new MacBook is outstanding."

# Perform sentiment analysis
result = sentiment_pipeline(sentence)[0]

# Printing label and confidence score
print(f"Sentiment: {result['label']}")
print(f"Confidence Score: {result['score']}")


Device set to use cpu


Sentiment: POSITIVE
Confidence Score: 0.9998302459716797


**BERT (Bidirectional Encoder Representations from Transformers)** uses an encoder architecture, which focuses on understanding context by looking at both the left and the right side of a word.

**GPT (Generative Pre-trained Transformer)** uses a decoder architecture, which generates text by looking backwards (typically left-to-right).

**Using pre-trained models is beneficial because:**

General Knowledge: They already learn rich linguistic patterns, semantic relationships, and context from vast amounts of text.

Training Time and Resources: It saves significant computational resources and training time.

Higher Performance with Less Data: Pre-trained models enable fine-tuning on small, domain-specific datasets while retaining their strong general understanding of language.